In [39]:
!pip install tensorflow  opencv-python mediapipe sklearn matplotlib pillow

In [40]:
pip install pysocks scikit-learn  

Note: you may need to restart the kernel to use updated packages.


In [30]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont
import datetime


In [31]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import load_model

In [32]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 
current_directory = os.getcwd()
# Actions that we try to detect
#actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ','ඔ','ඕ','ක්','ග්','ජ්','ට්','ද්','ණ්','ත්','ඩ්','න්','ප්','බ්','ම්','ය්','ර්','ල්','ව්','ස්','හ්'])
# length_of_array = len(actions)
# print(length_of_array)
actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ'])	
# actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ'])	
# actions = np.array(['ඊ','උ','ඌ','එ','ඒ'])
# Thirty videos worth of data
no_sequences = 90

# Each One Video  going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [33]:
label_map = {label:num for num, label in enumerate(actions)}

In [34]:
sequences, labels = [], []
# loop each action
for action in actions:
    # Loop through the each action sequence(videos)
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # this array will hold all the frame in one video
        window = []
        # Loop though the each frame 30(Frame)
        for frame_num in range(sequence_length):
            file_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))

            if os.path.exists(file_path):
                res = np.load(file_path)
            else:
                print(f"Error: File not found at {file_path}")
            # # using np.load function load the saved np array from folder (path,which action,which video number,which frame number)
            # res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # add loaded framee to window
            window.append(res)
        #    
        sequences.append(window)
        labels.append(label_map[action])

In [35]:
# shape should be 900(30 action 1 action has 30 videos) videos,30 frame each,126(21*3*2) key points in a frame
np.array(sequences).shape

(900, 30, 126)

In [36]:
np.array(labels).shape

(900,)

In [37]:
X = np.array(sequences)

In [38]:
y = to_categorical(labels).astype(int)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [40]:
from tensorflow.keras.models import Sequential
# Make us able to action detection
from tensorflow.keras.layers import LSTM, Dense
# Make us able to monitor our modle during taining
from tensorflow.keras.callbacks import TensorBoard

In [41]:
# Get the current date in YYYYMMDD format
current_date_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint_path = "checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Check if the checkpoint directory exists, if not, create it
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Check if a latest checkpoint exists
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,  # Path to save the model weights
    save_weights_only=True,     # Save only the model weights
    verbose=1,                  # Verbosity mode, 1 for progress bar
    save_freq=50          
)



In [42]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [47]:
# load Sequential api
model = Sequential()
# add 3 layers of lstm model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [44]:
# If a latest checkpoint exists, load the weights and continue training
# if latest_checkpoint:
#     # print(f"Loading weights from checkpoint: {latest_checkpoint}")
#     model.load_weights(latest_checkpoint)
# else:
#     if os.path.exists('action.h5'):
#         # Load the pre-trained model from the H5 file
#         model = tf.keras.models.load_model('action.h5')

#         # Get the model summary
#         model.summary()

        # # Get the model weights
        # model_weights = loaded_model.get_weights()
        # model = model.set_weights(model_weights)



In [48]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print(f"Test Accuracy: {accuracy:.2f}")

2/2 [==============================] - 1s 8ms/step - loss: 2.3001 - categorical_accuracy: 0.1333
Test Accuracy: 0.13


In [49]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback,cp_callback])

Epoch 1/2000
11/27 [===========>..................] - ETA: 0s - loss: 2.3069 - categorical_accuracy: 0.1193
Epoch 1: saving model to checkpoints\cp-0001.ckpt
27/27 [==============================] - 5s 28ms/step - loss: 2.3033 - categorical_accuracy: 0.1298
Epoch 2/2000
27/27 [==============================] - 1s 26ms/step - loss: 2.2386 - categorical_accuracy: 0.1754
Epoch 3/2000
 7/27 [======>.......................] - ETA: 0s - loss: 2.2299 - categorical_accuracy: 0.1518
Epoch 3: saving model to checkpoints\cp-0003.ckpt
27/27 [==============================] - 1s 29ms/step - loss: 2.0957 - categorical_accuracy: 0.2199
Epoch 4/2000
27/27 [==============================] - 1s 26ms/step - loss: 2.1211 - categorical_accuracy: 0.2211
Epoch 5/2000
 3/27 [==>...........................] - ETA: 0s - loss: 1.8631 - categorical_accuracy: 0.3021
Epoch 5: saving model to checkpoints\cp-0005.ckpt
27/27 [==============================] - 1s 28ms/step - loss: 1.8057 - categorical_accuracy: 0.3053


KeyboardInterrupt: 

In [50]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            48896     
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 10)                330       
                                                                 
Total params: 203690 (795.66 KB)
Trainable params: 203

In [51]:
res = model.predict(X_test)

2/2 [==============================] - 0s 7ms/step


In [52]:
print(res)

[[1.23917644e-05 4.22300473e-05 5.21674410e-05 1.86511807e-05
  7.00881938e-05 1.40073774e-02 3.02185833e-01 6.83611333e-01
  1.60575120e-09 5.98978245e-09]
 [7.10608531e-03 1.63936224e-02 1.35031505e-03 1.81348831e-03
  9.39454079e-01 1.83253065e-02 2.79203778e-05 3.72707203e-04
  1.51382023e-02 1.81367323e-05]
 [5.11916767e-07 7.29524103e-08 6.04281127e-01 3.68338883e-01
  5.11254999e-04 7.51612999e-04 8.56503320e-05 1.25873339e-04
  3.32367141e-03 2.25813575e-02]
 [2.83944863e-03 9.94355321e-01 6.99487117e-08 5.49337964e-10
  5.57591775e-05 2.59112706e-03 3.16595742e-06 1.55209840e-04
  4.21396571e-08 1.51636928e-12]
 [7.60594094e-06 1.82117176e-07 5.02575636e-01 2.96073616e-01
  2.11083633e-03 1.46363617e-03 1.72101200e-01 2.52167527e-02
  3.24237067e-06 4.47279075e-04]
 [1.66033613e-04 5.19568093e-05 4.41733986e-01 4.50703591e-01
  1.61842313e-02 1.17051452e-02 1.44213100e-03 1.98826846e-03
  3.30347046e-02 4.29899730e-02]
 [9.96765018e-01 3.23415711e-03 1.11250342e-09 1.38774374e

In [53]:
actions[np.argmax(res[4])]

'ඇ'

In [54]:
actions[np.argmax(y_test[4])]

'ඈ'

In [55]:
model.save('action.h5')

In [56]:
model = load_model('action.h5')

In [57]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [58]:
yhat = model.predict(X_train)

27/27 [==============================] - 1s 9ms/step


In [59]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [60]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[768,   0],
        [  3,  84]],

       [[767,   2],
        [  0,  86]],

       [[748,  23],
        [ 44,  40]],

       [[731,  44],
        [ 27,  53]],

       [[765,   3],
        [  5,  82]],

       [[761,  10],
        [  1,  83]],

       [[748,  20],
        [ 30,  57]],

       [[739,  30],
        [ 18,  68]],

       [[768,   0],
        [  1,  86]],

       [[768,   0],
        [  3,  84]]], dtype=int64)

In [61]:

accuracy_score(ytrue, yhat)

0.8456140350877193